# Extraction and EDA

This notebook extracts the data and performs analysis to test for feature availability.

In [97]:
import os
import glob
from typing import Dict, Union, List
import logging
from functools import reduce
from copy import deepcopy
import pandas as pd
import numpy as np
import re, string
from pandas.core.tools.datetimes import _guess_datetime_format_for_array

# set logger level
logging.basicConfig(level=logging.DEBUG)

def data_into_dict(
        filepath: Union[str, List], 
) -> Dict[str, pd.DataFrame]:   
    """Read data from paths into dictionary values
    This is an example of Google style.

    Args:
        filepath (Union[str, List]) : string literal of list of strings pointing to files for io
    Returns:
        file_d: dictionary of files as dataframes, with key as filename abbreviation
    """
    logging.info(f"Filepaths: {filepath}")
    # create file dictionary
    file_d = dict()
    for f in filepath:
        # get abbreviation for key
        fname_abbr =  os.path.split(f)[1].split(".")[0]
        logging.info(f"Reading {f} into key {fname_abbr}")
        # read dataframe into value
        file_d[fname_abbr] = pd.read_csv(f)
    return file_d


# read in data from dir
data_path = "../data/data/"
# group filenames by prefix
a_files = data_into_dict(glob.glob(os.path.join(data_path, "a__*")))
b_files = data_into_dict(glob.glob(os.path.join(data_path, "b__*")))

# consolidate dataframe groups into merged structure 
def merge_all_frames(
    frames: Dict, 
    on: str, 
    how: str, 
    rename_exclusions: List=[]
) -> pd.DataFrame:
    """Merge all frames in list into single dataframe 

    Args:
        frames (Dict) : dict of dfs with frame values to merge into single frame
        on (str) : column to merge on 
        how (str) : merge type
        rename (List) : list of columns to not rename
    Returns:
        frame_merged (pd.DataFrame): merged dataframe
    """
    frames = deepcopy(frames)
    # append df key names to columns to resolve conflicts in col names
    for frame in frames.items(): 
        frames[frame[0]] = frame[1].rename({col:f"{col}_{frame[0]}" \
            for col in frame[1] if (col!=on) and (col not in rename_exclusions)}, axis=1)

    # merge frames and set key value as conflicting column suffixes
    logging.info(f"Merging values of {frames.keys()}")
    frame_merged = reduce(
                lambda  left,right: pd.merge(left,right,on=[on],how=how), frames.values()
            )
        
    logging.info(f"Merged frames into one with columns {frame_merged.columns}")
    return frame_merged

a_frame = merge_all_frames(a_files, on="geo_id", how='outer', rename_exclusions=['vendor_id'])
b_frame = merge_all_frames(b_files, on="b_entity_id", how='outer')

a_frame = a_frame.set_index("vendor_id").reset_index()
b_frame = b_frame.set_index("b_entity_id").reset_index()

INFO:root:Filepaths: ['../data/data/a__geo.csv', '../data/data/a__company.csv']
INFO:root:Reading ../data/data/a__geo.csv into key a__geo
INFO:root:Reading ../data/data/a__company.csv into key a__company
INFO:root:Filepaths: ['../data/data/b__company.csv', '../data/data/b__hierarchy.csv', '../data/data/b__address.csv']
INFO:root:Reading ../data/data/b__company.csv into key b__company
INFO:root:Reading ../data/data/b__hierarchy.csv into key b__hierarchy
INFO:root:Reading ../data/data/b__address.csv into key b__address
/Users/cullenpaulisick/opt/anaconda3/envs/webscraping/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (11,15) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
INFO:root:Merging values of dict_keys(['a__geo', 'a__company'])
INFO:root:Merged frames into one with columns Index(['geo_id', 'zipcode_a__geo', 'is_primary_a__geo', 'latitude_a__geo',
       'longi

In [95]:
def type_compression(
    frame: pd.DataFrame, 
    numeric_proportion_threshold: float=0.5
) -> pd.DataFrame:
    """Compress types of values into a standard

    Args:
        frame (pd.DataFrame) : data frame to compress types
        numeric_proportion_threshold (float) : proportion of values in mixed type column
            that must pass numeric validation in order to convert that column to either float or int type
    Returns: 
        frame_comp (pd.DataFrame) : compressed type data
    """
    # get column types
    col_types = frame.dtypes.to_dict()
    # replace na value with string literal
    frame = frame.fillna("null")
    # set heuristic inline function for testing non-null string values
    non_null = lambda x: ('null' not in x.lower().replace(" ", "")) \
        and ('nan' not in x.lower().replace(" ", ""))\
        and ('notdefined' not in x.lower().replace(" ", ""))\
        and ('na' not in x.lower().replace(" ", ""))
    # convert object types to string values
    for col, datatype in col_types.items():
        logging.info(f"working on column {col}")
        frame[col] = frame[col].astype(str)
        # get all values in series that are not null
        non_null_index = frame[col].apply(non_null)
        non_null_s = frame[col][non_null_index]
        # create inline function for testing if data is numeric, only stripping dashes from left 
        isnum = lambda x: (x.lstrip("-").replace(".", "").isnumeric()) and (x.count(".")==1)
        isnum_s = non_null_s.apply(isnum)
        isnum_sum, isnum_true = isnum_s.value_counts().sum(), isnum_s.value_counts().get(True)
        # get proportion of numeric strings in non-null series
        # get proportion is numeric values are present
        if isnum_true:
            isnum_prop = float(isnum_true)/float(isnum_sum)
        else: 
            isnum_prop = 0
        logging.info(f"Numeric proportion for {col}: {isnum_prop}")
        # strip whitespace
        frame[col] = frame[col].str.strip()
        # check proprtion and convert based upon value
        if isnum_prop >= numeric_proportion_threshold:
            col_numeric = frame[col].apply(isnum)
            # check for int values among all numeric values
            int_check = frame[col][col_numeric]\
                .apply(lambda x: set(x.split(".")[-1]).issubset("0") or x.isdigit())
            if int_check.all():
                # set null type
                frame.loc[~col_numeric,col] = np.nan
                # set type as int, with float conversion first
                # to offset np.nan type data
                frame[col] = frame[col].astype("float")
                frame[col] = frame[col].astype("Int64")
                logging.info(f"Converted column {col} into Int64 type.")
            else: 
                # set null type
                frame.loc[~col_numeric,col] = np.nan
                # set type as float if non-int numeric values exist
                frame[col] = frame[col].astype("float")
                logging.info(f"Converted column {col} into float type.")
        # if non null values are not primarily numeric, then convert to lowercase
        # and remove non-alphanumeric chars
        else:
            # convert all to lowercase
            frame[col] = frame[col].apply(str.lower)
            # create regex pattern for keeping only alphanumeric characters
            frame.loc[non_null_index, col] = frame[col][non_null_index]\
                .apply(lambda x: re.sub(r'[^A-Za-z0-9 ]+', "", x))
            # create regex pattern for removing redundant whitespace
            frame.loc[non_null_index, col] = frame[col][non_null_index]\
                .apply(lambda x: re.sub(" +", " ", x))
            # convert all null values to nan
            frame.loc[~non_null_index,col] = np.nan
            logging.info(f"Converted column {col} into string type.")

    return frame



comp = type_compression(frame=a_frame)

INFO:root:working on column vendor_id
INFO:root:Numeric proportion for vendor_id: 0
INFO:root:Converted column vendor_id into string type.
INFO:root:working on column geo_id
INFO:root:Numeric proportion for geo_id: 0
INFO:root:Converted column geo_id into string type.
INFO:root:working on column zipcode_a__geo
INFO:root:Numeric proportion for zipcode_a__geo: 0
INFO:root:Converted column zipcode_a__geo into string type.
INFO:root:working on column is_primary_a__geo
INFO:root:Numeric proportion for is_primary_a__geo: 0
INFO:root:Converted column is_primary_a__geo into string type.
INFO:root:working on column latitude_a__geo
INFO:root:Numeric proportion for latitude_a__geo: 0.9874989934777357
INFO:root:Converted column latitude_a__geo into float type.
INFO:root:working on column longitude_a__geo
INFO:root:Numeric proportion for longitude_a__geo: 0.9874989934777357
INFO:root:Converted column longitude_a__geo into float type.
INFO:root:working on column elevation_a__geo
INFO:root:Numeric pr

In [75]:
frame = a_frame
col = "address1_a__company"
frame = frame.fillna("null")
non_null = lambda x: bool('null' not in x.lower().replace(" ", ""))\
        and ('nan' not in x.lower().replace(" ", ""))\
        and ('notdefined' not in x.lower().replace(" ", ""))\
        and ('na' not in x.lower().replace(" ", ""))
frame[col] = frame[col].astype(str)
frame[col] = frame[col].str.strip()
# convert all to lowercase
frame[col] = frame[col].apply(str.lower)
# create regex pattern for only alphanumeric characters


# cut all non-alphanumeric characters from strings

non_null_index = frame[col].apply(non_null)
non_null_s = frame[col][non_null_index]
frame.loc[non_null_index, col] = frame[col][non_null_index].apply(lambda x: re.sub(r'[^A-Za-z0-9 ]+', '', x))
frame.loc[non_null_index, col] = frame[col][non_null_index].apply(lambda x: re.sub(' +', ' ', x))
frame.loc[~non_null_index,col] = np.nan

In [96]:
comp

,vendor_id,geo_id,zipcode_a__geo,is_primary_a__geo,latitude_a__geo,longitude_a__geo,elevation_a__geo,state_a__geo,state_full_name_a__geo,area_code_a__geo,...,address1_a__company,address2_a__company,country_a__company,zipcode_a__company,parentdunsnumber_a__company,score_a__company,cnt_opp_a__company,bucket_id_a__company,load_date_a__company,lvl_a__company
0,285924451,8278,14467,t,43.043294,-77.614181,513,ny,new york,585,...,205 summit point dr 2,NaN,us,14467,774581862,71,0,87,20180526,0
1,47653720,3430,6258,t,41.897500,-71.963100,287,ct,connecticut,860,...,NaN,NaN,us,06258,<NA>,0,0,99,20180526,0
2,149196787,3430,6258,t,41.897500,-71.963100,287,ct,connecticut,860,...,junction rte 101 and rte 169,NaN,us,06258,<NA>,0,0,27,20180526,1
3,274892372,3430,6258,t,41.897500,-71.963100,287,ct,connecticut,860,...,NaN,NaN,us,06258,<NA>,0,0,20,20180526,1
4,197539987,3430,6258,t,41.897500,-71.963100,287,ct,connecticut,860,...,rt 101 po box 98 pomfret,NaN,us,06258,<NA>,0,0,48,20180526,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76339,49244286,17923,29334,t,34.908130,-82.117325,816,sc,NaN,864,...,2153,e main st 14,us,29334,<NA>,61,0,3,20180526,0
76340,21988338,4090,7430,t,41.077976,-74.176374,22,nj,new jersey,201,...,520 green mountain rd mahwah,NaN,us,07430,<NA>,0,0,89,20180526,0
76341,36866816,4090,7430,t,41.077976,-74.176374,22,nj,new jersey,201,...,10 industrial ave,NaN,us,07430,72704224,46,0,90,20180526,0
76342,63214740,4090,7430,t,41.077976,-74.176374,22,nj,new jersey,201,...,115 franklin tpke,NaN,us,07430,884074209,25,0,58,20180526,0


In [101]:
a_frame.load_date_a__company.to_date

0        2018-05-26
1        2018-05-26
2        2018-05-26
3        2018-05-26
4        2018-05-26
            ...    
76339    2018-05-26
76340    2018-05-26
76341    2018-05-26
76342    2018-05-26
76343    2018-05-26
Name: load_date_a__company, Length: 76344, dtype: object